# Dataset

### 이미지 불러오기

In [1]:
import os
import cv2


# 폴더 경로에 존재하는 이미지 파일을 읽어오는 함수
def load_images_from_folder(dir_path):
    images = []
    labels = []
    label = dir_path.split('/')[-1] # 'cat' 또는 'dog'
    for file_name in os.listdir(dir_path):
        image_path = os.path.join(dir_path, file_name) # 이미지 파일 경로
        image = cv2.imread(image_path) # 이미지 읽어오기
        image = cv2.resize(image, (32, 32)) # 이미지의 크기를 (32, 32)로 조정
        image = image.reshape(-1) # 이미지를 1차원으로 변경
        image = image / 255.0 # 데이터 정규화
        images.append(image)
        labels.append(label)

    return images, labels

In [2]:
cat_images, cat_labels = load_images_from_folder('../data/cifar10_images/cat')
dog_images, dog_labels = load_images_from_folder('../data/cifar10_images/dog')

print(f"이미지 모양: {cat_images[0].shape}")
print(f"고양이 데이터 갯수: {len(cat_labels)}")
print(f"강아지 데이터 갯수: {len(dog_labels)}")

이미지 모양: (3072,)
고양이 데이터 갯수: 500
강아지 데이터 갯수: 500


### 레이블 벡터화

In [3]:
import numpy as np

# 데이터와 레이블 병함
images = np.array(cat_images + dog_images)
labels = np.array(cat_labels + dog_labels)

# 레이블 -> 벡터
label_to_index = {'cat': 0, 'dog': 1}
indexed_labels = np.array([label_to_index[label] for label in labels])

print(f"벡터화된 레이블: {indexed_labels[:10]}")

벡터화된 레이블: [0 0 0 0 0 0 0 0 0 0]


### 학습 데이터와 테스트 데이터 나누기

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, indexed_labels, test_size=0.2, random_state=42)

# 손실 함수 사용을 위한 레이블 모양 변경
y_train = y_train.reshape((800, 1))
y_test = y_test.reshape((200, 1))

print(f"학습 데이터 갯수: {len(y_train)}")
print(f"테스트 데이터 갯수: {len(y_test)}")

학습 데이터 갯수: 800
테스트 데이터 갯수: 200


# HyperParameter

In [5]:
num_features = 3072 # 입력 데이터의 크기
num_samples = 800 # 학습 데이터의 갯수
learning_rate = 0.0001 # 스텝 크기
num_iterations = 1000 # 최적화 진행 횟수

# Model

In [6]:
# 선형 모델 초기화
W = np.random.randn(num_features, 1)
b = np.random.randn(1)

# Non-linear function

In [7]:
# 입력 받은 logit 값들의 범위를 0과 1 사이로 변형
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Loss

$
\begin{align}
-\frac{1}{N}\sum_{i=1}^N[y_ilog(p_i)+(1-y_i)log(1-p_i)]
\end{align}
$

1. 선형 모델의 출력: [0.5, -1.2, 0.8]
2. Sigmoid의 출력: [0.62245933, 0.23147522, 0.68997448]
3. 실제 레이블: [1, 0, 1]

In [8]:
# 이진 크로스 엔트로피 손실 함수
# 모델의 예측 확률과 실제 레이블 간의 차이를 측정하여, 예측이 정확할수록 손실을 줄이는 방향으로 모델을 학습시킵니다.
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Numerical stability
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

# Train

In [9]:
# 경사 하강법을 통한 학습
for i in range(num_iterations):
    # 선형 모델의 예측 계산
    z = np.dot(x_train, W) + b # Wx + b
    # 활성화 함수 Sigmoid 통과
    y_hat = sigmoid(z)
    
    # 손실 계산
    loss = binary_cross_entropy(y_train, y_hat)
    
    # 손실의 W 및 b에 대한 미분 계산 (Backpropagation)
    dW = -2 * np.dot(x_train.T, (y_train - y_hat)) / num_samples
    db = -2 * np.sum(y_train - y_hat) / num_samples
    
    # 매개변수 업데이트
    W -= learning_rate * dW
    b -= learning_rate * db
    
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {loss}")

# 최종 손실 출력
print(f"Final Loss: {loss}")

Iteration 0, Loss: 12.363720921718873
Iteration 100, Loss: 10.46257272067155
Iteration 200, Loss: 8.421303153236783
Iteration 300, Loss: 6.66053147675542
Iteration 400, Loss: 5.491611588435436
Iteration 500, Loss: 4.893676508633495
Iteration 600, Loss: 4.606196418950369
Iteration 700, Loss: 4.466338877698652
Iteration 800, Loss: 4.397349998007791
Iteration 900, Loss: 4.357801414376649
Final Loss: 4.330735984599587


# Predict

In [10]:
z = np.dot(x_test, W) + b

# 시그모이드 함수는 모델의 출력을 [0, 1] 사이의 확률 값으로 변환합니다.
y_hat_test = sigmoid(z)

# 확률이 0.5보다 큰 경우를 '1' (dog)로, 그렇지 않은 경우를 '0' (cat)로 변환합니다.
y_pred_test = (y_hat_test > 0.5).astype(int)

# 정확도를 계산합니다.
accuracy = np.mean(y_pred_test == y_test)

# 계산된 정확도를 백분율로 출력합니다.
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")


Test Set Accuracy: 53.50%
